## Data fields & description:
- Project Number: the unique project id
- Project Title: the project name (Mandatory)
- Project Title English: the project name in English (Optional)
- Reponsible Applicant: the person who submitted the research proposal to the SNSF (Mandatory)
- Funding Instrument: funding schemes for research and scientific communication
- Funding Instrument Hierarchy: top level hierarchy funding instrument
- Institution: the research institution where the project will largely be carried out
- University: the University where the project will largely be carried out. **This field is only filled if the research is carried out at a Swiss institution, otherwise the field remains blank. In the case of mobility fellowships, it is generally left empty.**
- Discipline Number: number of the discipline
- Discipline Name: name of the discipline
- Discipline Name Hierarchy: top level of discipline
- Start Date: the starting date of the project
- End Date: the actual end date of the project
- Approved Amount: the total approved money spent for the project. **This amount is not indicated in the case of mobility fellowships.**
- Keywords: unstructured field.

In [1]:
# import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML
import folium
import requests
%matplotlib inline

In [2]:
# Define constants
GEONAMES_SEARCH_URL = 'http://api.geonames.org/searchJSON'
GOOGLE_SEARCH_URL = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
GOOGLE_GEOCODE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

In [3]:
# load the p3
df = pd.read_csv('./data/P3_GrantExport.csv', sep=";")

The documentation of fields mentions that institutes from outside Switzerland don't have an entry in University. So it's safe to throw away all rows has 'nicht zuteilbar - NA' in university column.

In [4]:
df_Nicht = df[df['University'] == 'Nicht zuteilbar - NA']
df = df[df['University'] != 'Nicht zuteilbar - NA']

In [5]:
df_null = df[df['University'].isnull()]
df = df[df['University'].notnull()]

In [112]:
universities = list(df['University'].unique())
for university in universities:
    if ' - ' not in university:
        df.loc[df['University'] == university,'University'] = university[::-1].replace(" "," - ",1)[::-1]

Forschungskommission SAGW


## GeoNames Search Webservice:
GeoNames api offers services in XML and JSON format. We use JSON format as it is easier to extract data from it.

In [7]:
def search(url, params):
    """ This function returns a json file containing the geographical results from the
    Geonames or google map websites """
    try:
        response = requests.get(url, params)
    except:
        raise ServiceException(url, params)
    return response.json()

In [8]:
lst = []
for university in list_of_universities:
    params = {'q': university, 'maxRows': 1, 'country': 'CH', 'username': 'sorooshafiee'}
    response = search(GEONAMES_SEARCH_URL, params)
    lst.append(response['totalResultsCount'])
print('Number of the recovered contons from the university name: {}/{}'.format(sum(x != 0 for x in lst), len(lst)))

Number of the recovered contons from the university name: 5/76


## Google API


In [9]:
def get_geonames(url, params, types):
    """ This function returns the filtered address component """
    try:
        response = requests.get(url,params)
    except:
        raise ServiceException(url, params)
    print(response.json())
    address_comps = response.json()['results'][0]['address_components']
    filter_method = lambda x: len(set(x['types']).intersection(list(types)))
    return filter(filter_method, address_comps)

In [16]:
with open('data/api_key.txt', 'r') as in_file:
    api_key = in_file.read()

In [17]:
response = {}
for university in list_of_universities:
    params = {'key' : api_key, 'query' : university}
    response[university] = search(GOOGLE_SEARCH_URL, params)
    print(response[university]['status'], university)

OK Université de Genève - GE
ZERO_RESULTS NPO (Biblioth., Museen, Verwalt.) - NPO
OK Universität Basel - BS
OK Université de Fribourg - FR
OK Universität Zürich - ZH
OK Université de Lausanne - LA
OK Universität Bern - BE
ZERO_RESULTS Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL
OK Université de Neuchâtel - NE
OK ETH Zürich - ETHZ
OK Inst. de Hautes Etudes Internat. et du Dév - IHEID
OK Universität St. Gallen - SG
ZERO_RESULTS Weitere Institute - FINST
ZERO_RESULTS Firmen/Privatwirtschaft - FP
OK Pädagogische Hochschule Graubünden - PHGR
OK EPF Lausanne - EPFL
OK Pädagogische Hochschule Zürich - PHZFH
OK Universität Luzern - LU
OK Schweiz. Institut für Kunstwissenschaft - SIK-ISEA
OK SUP della Svizzera italiana - SUPSI
OK HES de Suisse occidentale - HES-SO
OK Robert Walser-Stiftung Bern - RWS
OK Paul Scherrer Institut - PSI
OK Pädagogische Hochschule St. Gallen - PHSG
OK Eidg. Anstalt für Wasserversorgung - EAWAG
OK Eidg. Material und Prüfungsanstalt - EMPA
ZERO_RESULTS Physikal.

In [ ]:
lst = []
types = ['administrative_area_level_1']
response = {}
for university in list_of_universities:
    params = {'key' : api_key, 'query' : university}
    response = search(GOOGLE_SEARCH_URL, params)
    try:
        location = response['results'][0]['geometry']['location']
        latlng = str(round(location['lat'],2)) + ',' + str(round(location['lng'],2))
    except:
        raise KeyError
    params = {'latlng': latlng}
    address_comps = get_geonames(GOOGLE_GEOCODE_URL, params, types)
    canton = list(address_comps)[0]['long_name']
    lst.append((university, canton))
    print(university, canton)
print('Number of the recovered universities conton: {}/{}'.format(sum(x != 0 for x in lst), len(lst)))

In [14]:
response[university]

{'error_message': 'You have exceeded your daily request quota for this API.',
 'html_attributions': [],
 'results': [],
 'status': 'OVER_QUERY_LIMIT'}

In [19]:
lst = []
for university in list_of_universities:
    
    if response[university]['status'] == 'ZERO_RESULTS':
        lst.append(university)

In [20]:
len(lst)

16

In [21]:
lst

['NPO (Biblioth., Museen, Verwalt.) - NPO',
 'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL',
 'Weitere Institute - FINST',
 'Firmen/Privatwirtschaft - FP',
 'Physikal.-Meteorolog. Observatorium Davos - PMOD',
 'Swiss Center for Electronics and Microtech. - CSEM',
 'Weitere Spitäler - ASPIT',
 'AO Research Institute - AORI',
 'Forschungsanstalten Agroscope - AGS',
 'Zürcher Fachhochschule (ohne PH) - ZFH',
 'Forschungsinstitut für Opthalmologie - IRO',
 'Forschungskommission SAGW',
 'Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP',
 'Schweizer Kompetenzzentrum Sozialwissensch. - FORS',
 'Pädagogische Hochschule Wallis - PHVS',
 'Staatsunabh. Theologische Hochschule Basel - STHB']

In [25]:
import re
m = re.search('(?<=-)\w+', 'Staatsunabh. Theologische Hochschule Basel - STHB').group(0)


AttributeError: 'NoneType' object has no attribute 'group'

In [24]:
m.group(0)

AttributeError: 'NoneType' object has no attribute 'group'

In [111]:
m = re.search(r'(.+?)\.\ (.+?) \-\ (\w+)', 'Theologische Hochschule Basel - STHB')
m.group(0)

AttributeError: 'NoneType' object has no attribute 'group'

In [60]:
match = re.search(r'(\w+), (\w+): (\S+)', 'Doe, John: 555-1212')

In [64]:
match.group(3)

'555-1212'

In [121]:
df.loc[df['University'] == 'Forschungskommission SAGW','University'] = 'Forschungskommission - SAGW'

In [124]:
df[df['University'] == 'Forschungskommission - SAGW']

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
31512,101005,SELECTS 2003: Eidgenössische Wahlen 2003,SELECTS 2003: Federal elections 2003,Malaguerra Carlo,Swiss Priority Programmes (SPPs),Programmes,Schweizerische Akademie der Geistes-und Sozial...,Forschungskommission - SAGW,10202,Political science,"Human and Social Sciences;Sociology, social wo...",01.02.2003,31.01.2004,100000.00,NaN
